In [ ]:
# 1. Extract Frames from Video

In [ ]:
# Extract frames as images from a video. In order to control performance and quality, we define 2 configurations:
# - chunk_duration (int - value in seconds)
# - frames_per_chunk (int)
# The video is then split into chunks of length chunk_duration. In the next step, we extract <frames_per_chunk> frames of each chunk into a jpeg file. The selected frames inside a chunk are evenly distributed across the overall amount of frames in that chunk if the number of frames per chunk is smaller than the total amount of frames in that chunk.

import cv2
import os
import math


def extract_frames(video_path, output_folder, chunk_duration, frames_per_chunk):
    # Open the video file
    vidcap = cv2.VideoCapture(video_path)
    if not vidcap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties (same as in attachment upload)
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_duration = total_frames / fps

    print(f"Video FPS: {fps}")
    print(f"Total Frames: {total_frames}")
    print(f"Video Duration: {video_duration}s")

    # Define number of chunks
    num_chunks = math.ceil(video_duration / chunk_duration)
    frames_per_second = fps
    print(f"Number of chunks: {num_chunks}")

    # Create output folder for jpg files
    os.makedirs(output_folder, exist_ok=True)

    count = 0
    for chunk_index in range(num_chunks):
        # Determine frame indices to extract for the current chunk to ensure even distribution
        start_time = chunk_index * chunk_duration
        end_time = min((chunk_index + 1) * chunk_duration, video_duration)
        start_frame = math.floor(start_time * frames_per_second)
        end_frame = math.ceil(end_time * frames_per_second)
        frames_in_chunk = end_frame - start_frame

        if frames_per_chunk > 0:
            if frames_per_chunk == 1:
                # Select only the middle frame of the chunk
                frame_indices = [start_frame + (frames_in_chunk // 2)] if frames_in_chunk > 1 else [start_frame]
            else:
                frame_indices = [
                    start_frame + round(i * (frames_in_chunk - 1) / max((frames_per_chunk - 1), 1))
                    for i in range(frames_per_chunk)
                ] if frames_in_chunk > 1 else [start_frame]
        else:
            frame_indices = []

        print(f"Chunk {chunk_index + 1}: Start Frame {start_frame}, End Frame {end_frame}")
        print(f"Frames to extract: {frame_indices}")

        # Read all frames from video and extract those we are interesed in
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        success, image = vidcap.read()
        current_frame = start_frame

        while success and current_frame < end_frame:
            if current_frame in frame_indices:
                output_filename = os.path.join(output_folder, f"chunk{chunk_index+1}_frame{current_frame}.jpg")
                cv2.imwrite(output_filename, image)
                print(f"Saved frame {current_frame} to {output_filename}")

            success, image = vidcap.read()
            current_frame += 1
    vidcap.release()

In [ ]:
# Example usage
video_path = 'gettyimages-142618122-640_adpp.mp4'
output_folder = './extracted_frames'
chunk_duration = 11  # seconds
frames_per_chunk = 1

extract_frames(video_path, output_folder, chunk_duration, frames_per_chunk)

# 2A. Transcript each frame to text

In [ ]:
from llama_index.multi_modal_llms.ollama import OllamaMultiModal
from llama_index.core import SimpleDirectoryReader

mm_model = OllamaMultiModal(model="llama3.2-vision")

for image_file in os.listdir(output_folder):
    # load as image documents
    image_documents = SimpleDirectoryReader(input_files=[os.path.join(output_folder, image_file)]).load_data()
    response = mm_model.complete(prompt="Provide a brief and condense description of what you can see in the image.", image_documents=image_documents)
    print(response)

# 2B. Transcript audio to text

# 3. Store everything to a multi-modal vector store with the video id for filtering